# Wrangle and Analyze Data

## Table of Contents
- [Introduction](#intro)
- [Gathering Data](#gather)
- [Assessing Data](#assess)
 - [Quality](#quality)
 - [Tidiness](#tidy)
- [Cleaning Data](#clean)
- [Analyzing Data](#analyze)
- [Conclusion](#conclusion)

<a id='intro'></a>
### Introduction
This project will illustrate the data wrangling process.  The dataset that is being wrangled comes from the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as WeRateDogs.  WeRateDogs is a Twitter account that rate's people's dogs with a humorous comment about the dog.  These ratings almost always have a denominator of 10 and numerators almost always greater than 10.  With this data, I will try to create interesting and trustworthy analyses and visualizations.

<a id = 'gather'></a>
### Gathering Data

There are 3 pieces of data required for this project.  The first one is the WeRateDogs Twitter archive.  This file is given to us and will be treated like an internal file.

The second piece of data is tweet image predictions.  The file is hosted on Udacity's servers and will be downloaded programmatically using the Requests library and the following URL: [https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv].

The third piece of data will be queried from Twitter's API.  Using the tweet ID's in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt.  Each tweet's JSON data should be written to its own line.  Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

<a id = 'assess'></a>
### Assessing Data
Detect and document at least eight quality issues and two tidiness issues.

<a id = 'quality'></a>
**Quality**

<a id = 'tidy'></a>
**Tidiness**

<a id = 'clean'></a>
### Cleaning Data

<a id = 'analyze'></a>
### Analyzing Data

<a id = 'conclusion'></a>
### Conclusion